# Sentiment Analysis

In [18]:
!pip install pandas
!pip install numpy
!pip install sklearn


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [41]:
import pandas as pd
import numpy as np
import unicodedata
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB # cuando es texto se usa la multinomial
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

In [20]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


La base de datos cuenta con 891 observaciones y con 3 variables, de las cuales dos están clasificadas como objetos y una como entera.

In [22]:
df.sample(10)

,package_name,review,polarity
844,com.hamropatro,its very helpful,1
222,com.supercell.clashofclans,this update sucks!!! i can't open the game an...,0
81,com.linkedin.android,unexpected! the new update doesn't let you ch...,0
520,com.dropbox.android,its not safe it has a link to gallery and fil...,0
761,com.shirantech.kantipur,aa nice,1
165,com.imangi.templerun2,awesome its nice but not compatible with my h...,0
653,com.uc.browser.en,excellent a excellent option if you are looki...,1
866,com.rovio.angrybirds,"great game, too much ads angry birds is fun a...",1
327,com.viber.voip,viber rating - lacking code on the android ph...,0
161,com.imangi.templerun2,awesomness this game is awesome but tricky wi...,1


Se observa que las dos primeras variables deberían ser string mientras que la última debería ser categórica, por lo que se procede a hacer las transformaciones.

In [23]:
df[df.select_dtypes('object').columns]=df[df.select_dtypes('object').columns].astype('string')
df['polarity']=df['polarity'].astype('category')

A continuación se eliminan algunos caracteres desconocidos que se detectaron en el texto.

In [24]:
df['review']=df['review'].str.strip() # elimina espacios al comienzo y al final de la oracion
df['review']=df['review'].str.lower() # lleva todo a minuscula

In [25]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [26]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True) # elimina caracteres repetidos mas de dos veces


In [27]:
df.sample(10)

,package_name,review,polarity
707,com.opera.mini.native,worse 1. startup become slow. its mini or not...,0
111,com.linkedin.android,updates poorly current update clears the cache...,0
392,com.google.android.talk,1 major annoyance in the new update show who i...,0
762,com.shirantech.kantipur,i love this app it is better than tv and old r...,1
431,com.facebook.orca,useful and well designed however glitchy the a...,0
574,jabanaki.todo.todoly,doesn't seem to work at all. no widget and doe...,0
724,com.opera.mini.native,i want the old theme back this theme is awful....,0
167,com.imangi.templerun2,how to unlock frozen in this update have no da...,0
810,org.mozilla.firefox,nice browser but.. there is no doubt that fire...,1
218,com.supercell.clashofclans,it was a great game . until the last update . ...,0


In [28]:
df.iloc[675,]

package_name                                    com.hamrokeyboard
review          loved it rzrl app rrlrrl rrsrzrlrrl r rrsrzrlr...
polarity                                                        1
Name: 675, dtype: object

In [29]:
X=df['review']
y=df['polarity']

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2007,stratify=y)

In [33]:
vec=CountVectorizer(stop_words='english')
X_train=vec.fit_transform(X_train).toarray()
X_test=vec.transform(X_test).toarray()

In [36]:
X_train.shape

(668, 3142)

In [39]:
nb=MultinomialNB()
nb.fit(X_train,y_train)
print('R score', nb.score(X_train,y_train))

R score 0.9580838323353293


In [40]:
y_predict=nb.predict(X_test)

In [42]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.83      0.90      0.87       146
           1       0.78      0.65      0.71        77

    accuracy                           0.82       223
   macro avg       0.81      0.78      0.79       223
weighted avg       0.81      0.82      0.81       223



Se observa que el ajuste en la muestra de prueba es bastante bueno para predecir cualquiera de las categorias, hasta para la minoritaria el recall es mayor al 60%.

In [ ]:
import pickle
filename = '/workspace/Naive-Bayes/models/finalized_model.sav'
pickle.dump(nb, open(filename, 'wb'))